In [17]:
# モジュールをインポート
import numpy as np
import pandas as pd
import re
import requests
import lxml.html as lh
import pyprind

In [2]:
# Excelファイルの読み出し
file_name = '/Users/shohei/Desktop/Shohei/Fresnel/Formation/Data/dataset_new.xlsx'
sheet_name = '2018-10-25'
excel_book = pd.ExcelFile(file_name)
df_old = excel_book.parse(sheet_name)

In [91]:
# 'VizieR'から主星の金属量と質量を抽出
def extract(content, before, after):
    start = content.find(before)+len(before)
    end = content.find(after)-1
    return content[start: end]

N_star = 16682
pbar = pyprind.ProgBar(N_star)
star = []
metal = []
Ms = []

for num in range(N_star):
    
    num += 1
    url = f'https://vizier.u-strasbg.fr/viz-bin/VizieR-5?-ref=VIZ5bd292f4575a&-out.add=.&-source=J/A%2bA/530/A138/catalog&recno={num}'
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    table = doc.xpath('//table')
    content = table[2].text_content()
    
    try:
        star.append(extract(content, 'Name', 'Other').strip())
        metal.append(float(extract(content, '[Fe/H]', '[Sun]')))
        Ms.append(float(extract(content, 'M.MLP', 'Msun')))
    except:
        continue

    pbar.update()

0% [############                  ] 100% | ETA: 06:47:43

KeyboardInterrupt: 

In [ ]:
# 抽出したリストを元にDFを作成
df = pd.DataFrame({'star': star,
                   'Fe/H_vizier': metal,
                   'Ms_vizier': Ms,
                  })
df.to_csv('/Users/shohei/Desktop/Shohei/Fresnel/Formation/Data/vizier.csv')